## Python code for iceberg detection in SAR data

Note that the paths will need to be changed to fit the individual folder structure

### Step 1: Import Required Python Libraries

In this step, we import all the Python modules necessary for processing and visualizing the SAR and optical imagery.

- `numpy` (`np`): Provides efficient array operations and mathematical functions.  
- `matplotlib.pyplot` (`plt`): Used for plotting and visualizing images or data.  
- `sys.exit`: Allows the script to stop execution at a specified point, useful for debugging or partial runs.  
- `rasterio`: Reads and processes georeferenced raster datasets (GeoTIFF, JP2, etc.).  
  - `rasterio.plot.show`: Quick visualization of raster images.  
  - `rasterio.windows.Window` and `from_bounds`: Extract specific sub-regions (windows) from raster files.  
- `scipy.ndimage`: Provides image processing functions, e.g., uniform filtering for iDPolRAD preprocessing.  
- `PIL.Image` (`Img`): Handles general image reading and saving.  
- `itertools`: Supports advanced iteration operations (optional, for loops and combinatorics).  
- `pandas` (`pd`): Handles tabular data, used here for managing file lists and metadata.  
- `imageio`: Alternative library for reading/writing images.  
- `os` and `shutil`: File and directory management.  
- `cv2` (OpenCV): Optional image processing functions, e.g., resizing or masking.  
- `tkinter`: Used here to retrieve the system DPI for saving images at a specific size.

> **Tip:** Make sure all libraries are installed (e.g., `pip install rasterio matplotlib pandas imageio opencv-python`) before running the notebook.  
> `rasterio` may need special installation on some systems; `pip install rasterio` is recommended if `conda` fails.


In [ ]:
## Import these necessary libraries 

import numpy as np #this is a module that has array functionality 
import matplotlib.pyplot as plt #graph plotting module
from sys import exit #allows you to put an exit() to stop your code in a certain place, good for debugging or only running part of a script
import rasterio #rasterio - did it with pip install rasterio - conda didn't seem to work?
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.windows import from_bounds
from scipy import ndimage
from PIL import Image as Img
import itertools
import pandas as pd #good for dataframes
import imageio
import os
import cv2
import shutil
import tkinter
root = tkinter.Tk()
my_dpi = root.winfo_fpixels('1i')#this is used for determining dpi in order to save png images to a desired size.

### Step 2: Define Global Variables and Parameters

Before processing the imagery, we set several global variables and hyperparameters:

1. **Input CSV file** (`filenames_df`)  
   This file contains the file paths for each satellite dataset at every file index, including:
   - HH-polarized SAR images
   - HV-polarized SAR images
   - Optical imagery (JP2 format)

2. **Number of image cutouts per row** (`number_of_images_to_examine_per_row`)  
   Determines how many sub-images (cutouts) are extracted along one row of the full image.  
   Total cutouts per image = `(number_of_images_to_examine_per_row)^2`.

3. **Landmask threshold** (`landmask_limit`)  
   Defines the cutoff between land and water/fast ice in meters.  
   This threshold is used to remove high-reflectance land features from the analysis. Default = 13 m.

4. **iDPolRAD filter window sizes**  
   - `trainwindow`: size of the training window (pixels) used to compute the local background average. Must be large enough to capture landscape context but small enough to avoid large-scale variability.  
     Example: `trainwindow = 57` pixels → 570 m if pixel size = 10 m.
   - `testwindow`: size of the testing window (pixels) used to compute the local test statistic. Typically corresponds roughly to the expected iceberg size.  
     Example: `testwindow = 1` pixel → 10 m if pixel size = 10 m.

> **References:** These window sizes follow the optimal parameters found by Soldal et al., 2019 and Marino et al., 2016. Adjusting these values can affect the sensitivity and specificity of the iDPolRAD detection.


In [ ]:
#global variables that can be changed.
filenames_df=pd.read_csv(r'C:\Users\Image_data\Training\filenames_test.csv')#csv file containing the pathways for the HH, HV and jp2 image files at each file index.
number_of_images_to_examine_per_row=20#this number determines how many cutouts we want to make in one image row, where (number_of_images_to_examine_per_row)**2 is the total number of cutouts produced.
landmask_limit=13#Sets the limit between land and sea/fast ice. 13 looks good compared with HH image but could go a bit higher maybe. 

#You may want to play around with the window values below - read the section in Soldal et al. 2019 and the original paper Marino et al. 2016 
#These current values are based on the optimal window values found by Soldal et al. 2019.
trainwindow=57#size of the training window - needs to be quite big relative to iceberg but not so big that there is a significant change in landscape within the window. Remember this is in pixels so if they are 10m pixels this will be trainwindow*10m in size
testwindow=1#size of test window which should be around the size of an iceberg

### Step 3: Prepare and Verify SAR and Optical Data Files

In this step, we loop over all satellite file indices to set up the corresponding SAR HH, SAR HV, and optical images, as well as the land mask. The script performs the following tasks:

1. **Define file paths** for each data type using the `filenames_df` table:
   - HH-polarized SAR (`SARimfile_HH_geotiffzoom`)
   - HV-polarized SAR (`SARimfile_HV_geotiffzoom`)
   - Optical imagery (`imfile`)
   - Land mask (`landmask`)

2. **Check that all required files exist** for each file index.  
   If any file is missing, processing stops for that index.

3. **Keep track of valid indices** in `file_indexes_with_data_list` for which all data exist.

4. **Create output directories** for storing generated images, if they do not already exist.  
   Each directory is named according to the file index and processing step.

This ensures that only valid datasets are processed and that all outputs are organized systematically.  

> Note: This setup is essential before performing SAR/optical analysis or applying the iDPolRAD filter, because missing files or unorganized outputs can cause downstream errors.


In [ ]:
Examine the SAR HH, SAR HV and optical data of each file index.

file_indexes_with_data_list=[]#keep track of file indexes with data
for file_index in filenames_df['file_index'].tolist():
    
    #############################################################################################################
    #Set up the files
    
    #below choose which file you want to open
    SARimfile_HH_geotiffzoom = f"{base_dir}/Processed/{filenames_df['HH_file_name'].tolist()[file_index]}"
    SARimfile_HV_geotiffzoom = f"{base_dir}/Processed/{filenames_df['HV_file_name'].tolist()[file_index]}"
    imfile = f"{base_dir}/Processed/{filenames_df['jp2_file_name'].tolist()[file_index]}"
    landmask = f"{base_dir}/Processed/{filenames_df['landmask_file_name'].tolist()[file_index]}"
    
    #if Geotiff/jpeg2000/land mask data for one of the satellite data files doesnt exist, stop this script.
    if not os.path.isfile(SARimfile_HH_geotiffzoom):
        print('+++++++++++++++File index {} has no HH data, stop processing data!+++++++++++++++.'.format(file_index))
        break
    if not os.path.isfile(SARimfile_HV_geotiffzoom):
        print('+++++++++++++++File index {} has no HV data, stop processing data!+++++++++++++++.'.format(file_index))
        break
    if not os.path.isfile(imfile):
        print('+++++++++++++++File index {} has no optical data, stop processing data!+++++++++++++++.'.format(file_index))
        break
    if not os.path.isfile(landmask):
        print('+++++++++++++++File index {} has no land mask data, stop processing data!+++++++++++++++.'.format(file_index))
        break
        
    #add file_index to list if data exists for file index
    file_indexes_with_data_list.append(file_index)
        
    #if an image folder doesnt already exist, then make a folder for storing the produced image files.
    if not os.path.exists(r'C:\Users\Image_data\file_index_{}-land_mask'.format(file_index)):
        os.makedirs(r'C:\Users\Image_data\file_index_{}-land_mask'.format(file_index))

### Step 4 — Load and Pre-process SAR and Optical Data

In this section we load the co-located Sentinel-1 (SAR) and Sentinel-2 (optical) imagery and prepare them for iDPolRAD filtering.

What this cell does:

### 1. Read optical and SAR files using rasterio
We open each image to access:

the pixel size (from the affine transform),

the geographic bounds,

the raw image arrays.

### 2. Compute intensity percentiles for optical and SAR data
Percentile clipping helps improve contrast and stabilise brightness scaling across scenes.
Here we compute:

0.15–99.85% percentiles for the optical image,

8–93% percentiles for the SAR iDPolRAD output (chosen empirically).

3. Extract a matching spatial cutout
Because Sentinel-1 and Sentinel-2 tiles differ in size and footprint, we extract a common area using rasterio.windows.from_bounds() so that SAR and optical images align in physical space.

### 3. Prepare arrays for iDPolRAD
We load the HH and HV SAR channels, then create empty arrays for:

the HV and HH training means (large window),

the HV testing mean (small window).

5. Apply boxcar filters for iDPolRAD
The iDPolRAD filter is defined as:

### 4. iDPolRAD Filter Equations

The iDPolRAD filter enhances contrast between icebergs and surrounding sea ice by comparing local cross-polarised (HV) backscatter to the surrounding co-polarised (HH) background. The detector is computed as:

$$
\Lambda = \frac{\langle |HV|^2 \rangle_\text{test} - \langle |HV|^2 \rangle_\text{train}}{\langle |HH|^2 \rangle_\text{train}}
$$

where:  

- $\langle \cdot \rangle_\text{test}$ is the spatial average over the testing window (small window, e.g., 1×1 pixel),  
- $\langle \cdot \rangle_\text{train}$ is the spatial average over the training window (larger window, e.g., 57×57 pixels).

Traditionally, a threshold $T_\Lambda$ is applied to $\Lambda$ to detect objects, similar to CFAR detectors. In this work, instead of thresholding, we use the filtered images as input to the CNN, allowing the network to learn features directly from the processed data.

Because $\Lambda$ is a ratio, some intensity information is lost. To restore the original intensity information, the final iDPolRAD-filtered image is computed as:

$$
I = \Lambda \cdot \langle |HV|^2 \rangle_\text{test}
$$

Positive values indicate pixels where the HV backscatter is stronger than the local HH background (typical of icebergs), whereas negative values indicate lower HV backscatter than the local HH background (typical of open water patches).


### 5. Cleanup
To manage memory efficiently, large intermediate arrays (original SAR tiles, filtered components, etc.) are deleted once no longer needed.

In [ ]:
dataset = rasterio.open(imfile)#open imfile as defined above in 1.
    gt = dataset.transform# this shows the Affine matrix which is the matrix that georeferences the image to the Earth coordinates. 
    pixelSizeX = gt[0]#pixel size x metres
    pixelSizeY =-gt[4]#pixel size y metres
    #print('pixel size x metres',pixelSizeX)#comment this out if you want.
    #print('pixel size y metres',pixelSizeY)#comment this out if you want.

    #show the left, bottom, right and top physical bounds of the full Geotiff/jpeg2000 file. uncomment this if you like.
    #print(dataset.bounds)
    
    #print('uncomment show dataset below if you want to see the whole image but as its large it makes the programme run slowly')
    #show(dataset, title='full georeferenced dataset from imfile')#displays the image, comment this out as it makes the programme run slowly as its a big image
    
    
    #determine the 0.15th and 99.8th percentiles for the Optical data from the jpeg2000 image.
    with rasterio.open(imfile) as optdataset:
        optdataset_full = optdataset.read(1)
    optical_percentile_lower = np.percentile(np.concatenate(optdataset_full, axis=0),0.15)
    optical_percentile_upper = np.percentile(np.concatenate(optdataset_full, axis=0),99.85)
    #delete the full optical dataset that we just loaded in to save memory, since we just finished performing the calculations.
    del optdataset_full
    
    #since the Geotiff image is much larger than the jpeg2000 image, we need to make a cutout of the Geotiff image to match the jpeg2000 image (based on the known physical coordinates in both images).
    with rasterio.open(imfile) as dataset_full:
        cutout=from_bounds(dataset_full.bounds[0], 
                           dataset_full.bounds[1], 
                           dataset_full.bounds[2], 
                           dataset_full.bounds[3], 
                           dataset_full.transform)#To cut out in Earth coordinates
    
    
    #determine the 0.15th and 99.8th percentiles for the SAR iDPolRAD data from the Geotiff image.
    #Read in HV polarised SAR image from Sentinel 2
    with rasterio.open(SARimfile_HV_geotiffzoom) as HVfull:
        HVsmall = HVfull.read(1, window=cutout)#same cutout as used above
    #Read in HH polarised SAR image from Sentinel 2
    with rasterio.open(SARimfile_HH_geotiffzoom) as HHfull:
        HHsmall = HHfull.read(1, window=cutout)#same cutout as used above

    #below sets up blank arrays the same size as the images above
    HVtrain=HVsmall*0.0#i.e. just take the HVsmall array and fill it with zeros
    HVtest=HVsmall*0.0
    HHtrain=HHsmall*0.0

    HHtrain2d=ndimage.uniform_filter(HHsmall**2., size=trainwindow)#takes the HH data squared and convolves it with a mean filter width of trainwindow
    HVtrain2d=ndimage.uniform_filter(HVsmall**2., size=trainwindow)#takes the HV data squared and convolves it with a mean filter width of trainwindow
    HVtest2d=ndimage.uniform_filter(HVsmall**2., size=testwindow)#takes the HV data squared and convolves it with a mean filter width of testwindow

    filtim2d=(HVtest2d-HVtrain2d)/HHtrain2d# this is now formula (1) from Soldal et al. 2019 e.g. the iDPolRAD filter
    idpolrad=filtim2d*HVtest2d
    
    sar_percentile_lower=5.#orig 0.15
    sar_percentile_upper=95.#orig 99.85
    
    
    sar_idpolrad_percentile_lower = np.percentile(np.concatenate(idpolrad, axis=0),sar_percentile_lower)
    sar_idpolrad_percentile_upper = np.percentile(np.concatenate(idpolrad, axis=0),sar_percentile_upper)
    #delete the full SAR iDPolRAD dataset that we just loaded in to save memory, since we just finished performing the calculations.
    del HHsmall, HVsmall
    del HHtrain, HVtrain, HVtest
    del HHtrain2d, HVtrain2d, HVtest2d
    del filtim2d
    del idpolrad

### Step 5: Determine Physical Coordinates for Image Slices

Before extracting image cutouts, we calculate the real-world coordinates for each slice based on the full image bounds. This allows us to make spatially accurate slices from both SAR and optical images.

1. **Compute Step Size**  
   - `x_physical_steps_per_image` and `y_physical_steps_per_image` represent the horizontal and vertical lengths of each image slice.  
   - These are determined by dividing the total image width and height (from `dataset.bounds`) by the number of slices per row (`number_of_images_to_examine_per_row`).

2. **Generate Slice Corner Coordinates**  
   - Bottom-left (`x_physical_coord_list_left_side`, `y_physical_coord_list_left_side`) and top-right (`x_physical_coord_list_right_side`, `y_physical_coord_list_right_side`) coordinates for each slice are calculated using the step sizes.  
   - This ensures that each slice corresponds to a distinct, non-overlapping region of the full image.

3. **Create Full Combinations**  
   - Using `itertools.product`, all possible combinations of x and y coordinates are generated for both bottom-left and top-right corners.  
   - This produces lists (`merged_x_physical_coord_list_left_side`, `merged_y_physical_coord_list_left_side`, etc.) that can be directly used to define each slice window in Earth coordinates.

These coordinate lists provide a geospatial reference for extracting and saving image cutouts in later steps.


In [ ]:
#Determine the coords for the different image windows

    #measure the physical horizontal length of the full Geotiff/jpeg2000 file to determine the size of a horizontal image slice step (based on the global variable).
    x_physical_initial = dataset.bounds[0]
    x_physical_end = dataset.bounds[2]
    x_physical_steps_per_image = (x_physical_end-x_physical_initial)/number_of_images_to_examine_per_row

    #measure the physical vertical length of the full Geotiff/jpeg2000 file to determine the size of a vertical image slice step (based on the global variable).
    y_physical_initial = dataset.bounds[1]
    y_physical_end = dataset.bounds[3]
    y_physical_steps_per_image = (y_physical_end-y_physical_initial)/number_of_images_to_examine_per_row

    #make lists of the x,y physical coordinates for the bottom left side of each image slice step.
    x_physical_coord_list_left_side = []
    y_physical_coord_list_left_side = []
    for j in range(number_of_images_to_examine_per_row):
        x_physical_coord_list_left_side.append(x_physical_initial+(x_physical_steps_per_image*j))
        y_physical_coord_list_left_side.append(y_physical_initial+(y_physical_steps_per_image*j))

    #make lists of the x,y physical coordinates for the top right side of each image slice step.
    x_physical_coord_list_right_side = []
    y_physical_coord_list_right_side = []
    for j in range(1,number_of_images_to_examine_per_row+1):
        x_physical_coord_list_right_side.append(x_physical_initial+(x_physical_steps_per_image*j))
        y_physical_coord_list_right_side.append(y_physical_initial+(y_physical_steps_per_image*j))
        
    #merge the lists of all possible x,y physical coordinates combinations for the bottom left side of each image slice step into one list.
    combined_physical_coord_list_left_side = list(itertools.product(x_physical_coord_list_left_side, y_physical_coord_list_left_side))
    merged_x_physical_coord_list_left_side = [i[0] for i in combined_physical_coord_list_left_side]
    merged_y_physical_coord_list_left_side = [i[1] for i in combined_physical_coord_list_left_side]

    #merge the lists of all possible x,y physical coordinates combinations for the top right side of each image slice step into one list.
    combined_physical_coord_list_right_side = list(itertools.product(x_physical_coord_list_right_side, y_physical_coord_list_right_side))
    merged_x_physical_coord_list_right_side = [i[0] for i in combined_physical_coord_list_right_side]
    merged_y_physical_coord_list_right_side = [i[1] for i in combined_physical_coord_list_right_side]
    

### Step 6: Extract Georeferenced Cutouts Using Rasterio Windows

To save memory and handle large SAR and optical images efficiently, we extract smaller "cutouts" from the full image using `rasterio.windows.from_bounds`.  

**Key steps:**

1. **Check for existing cutouts**  
   - Before processing, we check whether a PNG output already exists for a slice to avoid redundant computation.

2. **Define the cutout window**  
   - Using the merged coordinate lists (`merged_x_physical_coord_list_left_side`, etc.), a `from_bounds` window is created for the slice of interest.  
   - This window corresponds to the exact georeferenced physical coordinates of that slice.

3. **Read only the window**  
   - `dataset_full.read(1, window=cutout)` reads only the pixels in the window rather than the entire image, saving memory.  
   - `dataset_full.window_transform(cutout)` retrieves the Affine transform for the cutout, giving the pixel size and enabling georeferenced operations if needed.

4. **Notes on window data**  
   - The resulting `dataset_w` is a plain 2D NumPy array without georeferencing metadata.  
   - To visualize or save it as a georeferenced image, we can write it back to a GeoTIFF using the window's transform.

This approach allows us to efficiently process large geospatial datasets while preserving the spatial alignment between SAR and optical images.


In [ ]:
#Make a rasterio window that just reads a small cutout of the large image file and therefore saves on memory.

    #make an empty list to store the SAR iDPolRAD png image filename from each image slice step.
    SAR_iDPolRAD_images_filenames_list = []
    #run a for loop to make SAR iDPolRAD and Optical images at each image slice step.
    for slice_index in range(number_of_images_to_examine_per_row**2):
        
        #append the SAR iDPolRAD png image filename (see bottom section of step 7) for each image slice step to a new list.
        SAR_iDPolRAD_images_filenames_list.append(r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        
        #in case this script crashes, we will have to rerun entire script, this line will ignore images that have already been made.
        if os.path.isfile(r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(file_index,slice_index)):
            continue
        
        #use rasterio and the combined x,y physical coordinates list to obtain a cutout dataset of the Optical image data at each image slice step (based on the matching georeferenced coordinates from the full Geotiff/jpeg2000 data).
        with rasterio.open(imfile) as dataset_full:
            cutout=from_bounds(merged_x_physical_coord_list_left_side[slice_index], 
                               merged_y_physical_coord_list_left_side[slice_index], 
                               merged_x_physical_coord_list_right_side[slice_index], 
                               merged_y_physical_coord_list_right_side[slice_index], 
                               dataset_full.transform)#To cut out in Earth coordinates
            dataset_w = dataset_full.read(1,window=cutout)#Cut out in georeferenced coordinates - i.e. the bounds are the georeferenced boundaries in metres.
            win_transform = dataset_full.window_transform(cutout)#tells you the Affine matrix of the window
            #dataset_full is just the same as dataset above but here is not fully read in only the window dataset_w is fully read in so this saves memory

        pixelSizeX = win_transform[0]#pixel size of window x metres
        pixelSizeY =-win_transform[4]#pixel size of window y metres
        #print('pixel size of window x metres',pixelSizeX)#comment this out if you want.
        #print('pixel size of window y metres',pixelSizeY)#comment this out if you want.


        


### Step 7: Compute the iDPolRAD Filter and Apply Land Mask

The iDPolRAD filter (Soldal et al., 2019) enhances icebergs in SAR imagery by comparing the HV and HH polarizations across a small test window and a larger training window. 

**Key steps:**

1. **Read SAR cutouts**  
   - Extract the HV and HH polarization data for the current window using `rasterio.read()` with the `window` parameter.  
   - The Affine transform of the cutout is also obtained for saving georeferenced outputs.

2. **Prepare arrays for filtering**  
   - Initialize blank arrays for `HVtest`, `HVtrain`, and `HHtrain`.
   - Compute the spatial averages using `ndimage.uniform_filter` over the training and testing windows:
     - `trainwindow` is typically large relative to iceberg size (e.g., 57 pixels).  
     - `testwindow` corresponds roughly to a single iceberg (e.g., 1 pixel).

3. **Compute the iDPolRAD filter**  
   
   - This enhances features with strong HV scattering relative to the local HH background, highlighting potential icebergs.

4. **Save the filtered image**  
   - The iDPolRAD image is written as a GeoTIFF to preserve georeferencing for visualization and further processing.

5. **Read the optical cutout and land mask**  
   - Optical image cutouts are aligned using the same `window` coordinates.  
   - The land mask is applied to both SAR and optical images using thresholding, ensuring that land areas are excluded in further analysis.

This processing step produces georeferenced, filtered SAR images ready for iceberg detection and comparison with optical data.


In [ ]:
 #Recreate equation (1) from Soldal et al. 2019. This will create an iDPolRAD filter image which enhances the icebergs

        #Read in HV polarised SAR image from Sentinel 2
        with rasterio.open(SARimfile_HV_geotiffzoom) as HVfull:
            HVsmall = HVfull.read(1, window=cutout)#same cutout as used above
            HVsmall_transform = HVfull.window_transform(cutout)
        #Read in HH polarised SAR image from Sentinel 2
        with rasterio.open(SARimfile_HH_geotiffzoom) as HHfull:
            HHsmall = HHfull.read(1, window=cutout)#same cutout as used above
            HHsmall_transform = HHfull.window_transform(cutout)

        #below sets up blank arrays the same size as the images above
        HHtrain=HHsmall*0.0#i.e. just take the HVsmall array and fill it with zeros
        HVtrain=HVsmall*0.0
        HVtest=HVsmall*0.0

        HHtrain2d=ndimage.uniform_filter(HHsmall**2., size=trainwindow)#takes the HH data squared and convolves it with a mean filter width of trainwindow
        HVtrain2d=ndimage.uniform_filter(HVsmall**2., size=trainwindow)#takes the HV data squared and convolves it with a mean filter width of trainwindow
        HVtest2d=ndimage.uniform_filter(HVsmall**2., size=testwindow)#takes the HV data squared and convolves it with a mean filter width of testwindow

        filtim2d=(HVtest2d-HVtrain2d)/HHtrain2d# this is now formula (1) from Soldal et al. 2019 e.g. the iDPolRAD filter
        idpolrad=filtim2d*HVtest2d

        #display the different steps, not georeferenced, comment out if not needed
        #show(HVsmall, title='unprocessed SAR HV image')#unprocessed HV image
        #show(HHsmall, title='unprocessed SAR HH image')#unprocessed HH image
        #show(HHtrain2d, title='SAR HH^2 and convolved with training window')#squared and convolved with mean filter trainwindow
        #show(HVtrain2d, title='SAR HV^2 and convolved with training window')#squared and smoothed
        #show(HVtest2d, title='SAR HV^2 and convolved with test window')#squared and smoothed
        #show(filtim2d, title='SAR iDPolRAD')#iDPolRAD filter

        filterfile=r'C:\Users\file_index_{0}-land_mask\Soldal_HHHV_Filter_for_file_index_{0}_for_slice_index_{1}.tiff'.format(
            file_index,slice_index)#its probably worth saviing this and just loading it in again as the processing step to do the iDPolRAD filter takes a while.

        filterdataset = rasterio.open(filterfile,'w',driver='GTiff',height=idpolrad.shape[0],width=idpolrad.shape[1],count=1,dtype=idpolrad.dtype,crs='EPSG:32640',transform=HVsmall_transform)
        #you could use temp.tiff for all of them such that its only one file that keeps being made read in and overwritten rather than lots of files clogging up the directory.
        filterdataset.write(idpolrad, 1)
        filterdataset.close()
        
        #now open the file you made as 
        filterdataset = rasterio.open(filterfile)
        filterdataset_w = filterdataset.read(1)

        #below just showing the optical image now to show that there's icebergs where the bright spots are in the filtered SAR image
        with rasterio.open(imfile) as optdataset:
            optdataset_w = optdataset.read(1,window=cutout)#open imfile as a cutout
            
        
        #process the landmask for the image slice step.
        with rasterio.open(landmask) as landmask_data:
            landmask_data_arr = landmask_data.read(1,window=cutout)#reads channel 1 and turns it into an np.array - i.e. no longer georeferenced
        
        landmask_data_arr_mask = landmask_data_arr*0.#makes an array the same size as dataset_arr to use as mask

        landmask_data_arr_mask[np.where((landmask_data_arr >= landmask_limit) | (landmask_data_arr <= 1.))]=0.#masks land. the or statement about <1 is because there is a region of very low values to bottom right which I think is where data is just missing. 
        landmask_data_arr_mask[np.where((landmask_data_arr < landmask_limit) & (landmask_data_arr > 1.))]=1
        
        #add landmask to the SAR image.
        filterdataset_w = cv2.bitwise_and(filterdataset_w,filterdataset_w, 
                                          mask=landmask_data_arr_mask.astype(np.uint8))
        
        #add landmask to the optical image.
        optdataset_w = cv2.bitwise_and(optdataset_w,optdataset_w, 
                                          mask=landmask_data_arr_mask.astype(np.uint8))


### Step 8: Generate PNG Images of SAR iDPolRAD and Optical Cutouts

After computing the iDPolRAD filtered SAR image and reading the corresponding optical cutout:

1. **Compare SAR and Optical images side by side**  
   - A combined figure is created using `matplotlib.pyplot.subplots()` with two rows:
     - Top: Optical image  
     - Bottom: SAR iDPolRAD image  
   - Colorbars are added, and percentile-based scaling (`vmin`, `vmax`) is applied to enhance contrast for iceberg detection.  
   - The combined figure is saved as a PNG for visual inspection.

2. **Create individual PNG images**  
   - Optical and SAR iDPolRAD cutouts are saved separately as PNG images.  
   - Axes are removed (`plt.axis('off')`) for a clean image.  
   - Images are resized to the original cutout dimensions to compensate for cropping from `bbox_inches='tight'`.  
   - The images are saved in RGB format to ensure compatibility with image processing pipelines.

3. **File naming and organization**  
   - Each image slice step is saved with filenames that include:
     - `file_index` (the scene index)
     - `slice_index` (the cutout step)
   - This allows easy tracking and retrieval of specific image slices.

This step produces a complete set of preprocessed, visually comparable image cutouts for both SAR and optical data, which will be used for manual inspection, annotation, and eventual CNN training.


In [ ]:
#8Make png images of the cutout SAR iDPolRAD and Optical data cutouts at each image slice step.

        #make a combined plot to directly compare icebergs seen in the SAR iDPolRAD and Optical images.
        fig, (axColour,axFilter) = plt.subplots(2,1,figsize = (40,30))
        im1 = axColour.imshow(optdataset_w, 
                              vmin=optical_percentile_lower,
                              vmax=optical_percentile_upper)#displays the georeferenced optical image
        im2 = axFilter.imshow(filterdataset_w,
                              vmin=sar_idpolrad_percentile_lower,
                              vmax=sar_idpolrad_percentile_upper)#displays the georeferenced iDPolRAD image as in Soldal 2019 Fig 6. b.
        axColour.title.set_text('Optical image')
        axColour.title.set_fontsize(30)
        axFilter.title.set_text('SAR iDPolRAD image')
        axFilter.title.set_fontsize(30)
        axColour.tick_params(axis='both', which='major', labelsize=20)
        axFilter.tick_params(axis='both', which='major', labelsize=20)
        cb1 = plt.colorbar(im1, ax=axColour)
        cb2 = plt.colorbar(im2, ax=axFilter)
        plt.savefig(
            r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_and_Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index),bbox_inches='tight',facecolor='white', transparent=False)
        plt.close()
        
        #make a png image of the cutout Optical data at current image slice step.
        fig, ax = plt.subplots(1,figsize=(len(optdataset_w)/my_dpi, len(optdataset_w)/my_dpi),dpi=my_dpi)
        im1 = ax.imshow(optdataset_w,
                        vmin=optical_percentile_lower,
                        vmax=optical_percentile_upper)
        plt.axis('off')
        plt.savefig(
            r'C:\Users\Image_data\file_index_{0}-land_mask\Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index),bbox_inches='tight',pad_inches=0,dpi=my_dpi)
        plt.close()

        #resize the png image (we have to resize to original size because bbox_inches crops the white border around the image caused by removing axis in above step but doesnt alter the size of the image back to its original size)
        #and save the png image as RGB format rather than RGBA format (which is what plt.savefig() saves it as in the previous step).
        img = Img.open(r'C:\Users\Image_data\file_index_{0}-land_mask\Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        out = img.resize((len(optdataset_w),len(optdataset_w)))
        out.save(r'C:\Users\Image_data\file_index_{0}-land_mask\Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        img = Img.open(
            r'C:\Users\Image_data\file_index_{0}-land_mask\Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        background = Img.new(mode="RGB", size=(len(optdataset_w),len(optdataset_w)), color=(255, 255, 255))
        background.paste(img, mask = img.split()[3])
        background.save(r'C:\Users\Image_data\file_index_{0}-land_mask\Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index), "PNG", quality=100)

        
        #make a png image of the cutout SAR iDPolRAD data at current image slice step.
        fig, ax = plt.subplots(1,figsize=(len(optdataset_w)/my_dpi, len(optdataset_w)/my_dpi),dpi=my_dpi)
        im1 = ax.imshow(filterdataset_w,
                   vmin=sar_idpolrad_percentile_lower,
                   vmax=sar_idpolrad_percentile_upper)
        plt.axis('off')
        plt.savefig(
            r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index),bbox_inches='tight',pad_inches=0,dpi=my_dpi)
        plt.close()

        #resize the png image (we have to resize to original size because bbox_inches crops the white border around the image caused by removing axis in above step but doesnt alter the size of the image back to its original size)
        #and save the png image as RGB format rather than RGBA format (which is what plt.savefig() saves it as in the previous step).
        img = Img.open(r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        out = img.resize((len(optdataset_w),len(optdataset_w)))
        out.save(r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        img = Img.open(
            r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index))
        background = Img.new(mode="RGB", size=(len(optdataset_w),len(optdataset_w)), color=(255, 255, 255))
        background.paste(img, mask = img.split()[3])
        background.save(r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(
            file_index,slice_index), "PNG", quality=100)     

### Step 9: Record Top-Left Coordinates of Each Image Slice

Once the SAR iDPolRAD PNG images are generated, we compute the **top-left x and y physical coordinates** for each image slice. These coordinates allow us to:

1. Track the **physical location** of each cutout within the full georeferenced image.
2. Enable easy conversion between **pixel coordinates** and **real-world coordinates**, which is essential for mapping detected icebergs back to their true positions.

Steps performed in this code:

- Iterate over all slices in both horizontal and vertical directions to generate top-left coordinate lists.
- Combine the image filenames with their corresponding top-left coordinates into a `pandas.DataFrame`.
- Save the resulting DataFrame as a CSV file for future reference, ensuring each slice can be linked to its physical location in the original dataset.

This structured information is essential for dataset organization, manual inspection, and subsequent CNN training.


In [ ]:
 #Identify the top left x,y physical coordinates in each image slice step.

    #Make lists of the top left x,y physical coordinates in each image slice step.
    x_physical_coord_list_top_left = []
    y_physical_coord_list_top_left = []
    for j in range(number_of_images_to_examine_per_row):
        for k in range(number_of_images_to_examine_per_row):
            x_physical_coord_list_top_left.append(x_physical_coord_list_left_side[j])
            y_physical_coord_list_top_left.append(y_physical_coord_list_left_side[k] + y_physical_steps_per_image)   

    #Create a dataframe from the lists of the SAR iDPolRAD png image filenames and the top left x,y physical coordinates in each image slice step
    combined_physical_coord_df_top_left = pd.DataFrame(data={'filename':SAR_iDPolRAD_images_filenames_list,
                                           'x_coord_pixel_top_left_per_image_slice':x_physical_coord_list_top_left,
                                           'y_coord_pixel_top_left_per_image_slice':y_physical_coord_list_top_left})
    
    #save the dataframe to a new file.
    combined_physical_coord_df_top_left.to_csv(r'C:\Users\Image_data\combined_physical_coord_df_top_left.csv',
                                               index=False)
    

### Step 10: Clean Up Temporary Files

After generating the required SAR iDPolRAD and optical image slices for the current file index, temporary files created during processing are no longer needed.  

This step ensures:

- **Disk space is conserved** by removing intermediate files.
- **Workspace remains organized**, avoiding clutter from temporary TIFF and PNG files.
- Only the essential processed images and CSVs are retained for future analysis or CNN training.

Specifically, the script deletes:

1. Temporary PNG images used for intermediate visualization.
2. Temporary TIFF files created for georeferenced cutouts during processing.


In [ ]:
 #9. delete files that are no longer needed which were made as part of this file index.
    
    #delete the individual temp tiff file as it is no longer needed.
    if os.path.exists('..\Image_data\file_index_{0}-land_mask\temp.tiff'.format(file_index)):
        os.remove('..\Image_data\file_index_{0}-land_mask\temp.tiff'.format(file_index))
        

### Step 10: Organize Processed Images for Manual Labeling

Once all optical and SAR iDPolRAD image slices have been generated:

1. **Separate folders** are created to store all optical and all SAR images:
   - `all_optical_images-land_mask_test`
   - `all_SAR_iDPolRAD_images-land_mask_test`

2. The script **copies the processed images** from each file index subfolder into the respective master folder.  

This organization ensures that:

- Images are **centralized** for easy access during manual iceberg labeling with tools like `labelImg`.
- File management is simplified, avoiding the need to navigate multiple subfolders.
- Both optical and SAR images for the same geographic regions are **co-located**, facilitating direct visual comparison.


In [ ]:
#We will make copies and transfer all the produced optical and SAR images into these folders.
#We will use the images in these folders to compare optical and SAR images during our manual search for icebergs with labelImg.

#if a optical image folder doesnt already exist, then make a folder for storing only the produced optical image files.
if not os.path.exists(r'C:\Users\Image_data\all_optical_images-land_mask'):
    os.makedirs(r'C:\Users\Image_data\all_optical_images-land_mask')
#if a SAR image folder doesnt already exist, then make a folder for storing only the produced SAR image files.
if not os.path.exists(r'C:\Users\Image_data\all_SAR_iDPolRAD_images-land_mask'):
    os.makedirs(r'C:\Users\Image_data\all_SAR_iDPolRAD_images-land_mask')
    
#put a copy of the produced SAR and optical images into the new optical and SAR image folders.
for file_index in filenames_df['file_index'].tolist():
    
    #only copy images if data exists for the file index
    if file_index in file_indexes_with_data_list:
    
        #transfer all image slices to new folders
        for slice_index in range(number_of_images_to_examine_per_row**2):
            shutil.copy(r'C:\Users\Image_data\file_index_{0}-land_mask\Optical_image_for_file_index_{0}_for_slice_index_{1}.png'.format(file_index,slice_index),
                        r'C:\Users\Image_data\all_optical_images-land)_mask')
            shutil.copy(r'C:\Users\Image_data\file_index_{0}-land_mask\SAR_iDPolRAD_image_for_file_index_{0}_for_slice_index_{1}.png'.format(file_index,slice_index),
                        r'C:\Users\Image_data\all_SAR_iDPolRAD_images-land_mask')


### Training the YOLOv8 Model

Once the dataset is prepared, training is performed using YOLOv8's `train` function.  
For this tutorial, we focus on dataset preparation and visualization; the YOLOv8 training step is straightforward and can be executed with the official Ultralytics API.
